In [2]:
import openai
import os
import dotenv
from pathlib import Path
from llama_index.readers.file import PDFReader
from llama_index.core.response.notebook_utils import display_source_node
from llama_index.core.retrievers import RecursiveRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core import VectorStoreIndex, ServiceContext
from llama_index.llms.openai import OpenAI
from llama_index.core import Document
from llama_index.core.node_parser import SimpleNodeParser
from llama_index.core.schema import IndexNode

In [3]:
dotenv.load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

In [5]:
loader = PDFReader()
docs0 = loader.load_data(file=Path("G:\\Python\\Advance_RAG\\data\\EBOOK_BOT_TRADING.pdf"))

In [6]:
doc_text = "\n\n".join([d.get_content() for d in docs0])
docs = [Document(text=doc_text)]

In [8]:
node_parser = SimpleNodeParser.from_defaults(chunk_size=1024)
base_nodes = node_parser.get_nodes_from_documents(docs)

In [10]:
# set node ids to be a constant
for idx, node in enumerate(base_nodes):
    node.id_ = f"node-{idx}"

In [11]:
sub_chunk_sizes = [128, 256, 512]
sub_node_parsers = [
    SimpleNodeParser.from_defaults(chunk_size=c, chunk_overlap=50) for c in sub_chunk_sizes
]

all_nodes = []
for base_node in base_nodes:
    for n in sub_node_parsers:
        sub_nodes = n.get_nodes_from_documents([base_node])
        sub_inodes = [
            IndexNode.from_text_node(sn, base_node.node_id) for sn in sub_nodes
        ]
        all_nodes.extend(sub_inodes)

    # also add original node to node
    original_node = IndexNode.from_text_node(base_node, base_node.node_id)
    all_nodes.append(original_node)

In [12]:
all_nodes_dict = {n.node_id: n for n in all_nodes}

In [13]:
from llama_index.embeddings.openai import OpenAIEmbedding

embed_model = OpenAIEmbedding(embed_batch_size=10)
llm = OpenAI(model="gpt-4o")
service_context = ServiceContext.from_defaults(
    llm=llm, embed_model=embed_model
)


In [20]:
vector_index_chunk = VectorStoreIndex(
    all_nodes, service_context=service_context
)

C:\Users\PC Market\AppData\Local\Temp\ipykernel_18632\3567994959.py:6: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(


In [25]:
vector_retriever_chunk = vector_index_chunk.as_retriever(similarity_top_k=5)

In [26]:
retriever_chunk = RecursiveRetriever(
    "vector",
    retriever_dict={"vector": vector_retriever_chunk},
    node_dict=all_nodes_dict,
    verbose=True,
)

In [27]:
query_engine_chunk = RetrieverQueryEngine.from_args(
    retriever_chunk, service_context=service_context
)

In [28]:
response = query_engine_chunk.query(
    "Tổng hợp các thuật ngữ cơ bản được nhắc đến?"
)
print(str(response))

Retrieving with query id None: Tổng hợp các thuật ngữ cơ bản được nhắc đến?
Retrieved node with id, entering: node-29
Retrieving with query id node-29: Tổng hợp các thuật ngữ cơ bản được nhắc đến?
Retrieved node with id, entering: node-26
Retrieving with query id node-26: Tổng hợp các thuật ngữ cơ bản được nhắc đến?
Retrieved node with id, entering: node-42
Retrieving with query id node-42: Tổng hợp các thuật ngữ cơ bản được nhắc đến?
Retrieved node with id, entering: node-46
Retrieving with query id node-46: Tổng hợp các thuật ngữ cơ bản được nhắc đến?
Các thuật ngữ cơ bản được nhắc đến bao gồm:

1. **Trailing Up, Trailing Down, Stop Loss, Take Profit**: Các tùy chọn bổ sung giúp đảm bảo thu nhập hoặc ngăn ngừa thua lỗ.
2. **Bot profit**: Tổng lợi nhuận đạt được từ khi chạy bot cho đến hiện tại.
3. **Avg. daily profit**: Lợi nhuận và phần trăm lợi nhuận trung bình bot kiếm được hàng ngày.
4. **Unrealized**: Lãi/lỗ danh nghĩa của số vốn mà bot sử dụng ở giá hiện tại.
5. **Base currency